**SVM for image classification.** 

# importing necessary libraries

In [75]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from numpy import mean
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix


# loading CIFAR-10 dataset

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict

In [3]:
file = r'data_batch_1'
Data_batch_1 = unpickle(file)

In [4]:
type(Data_batch_1)

dict

In [5]:
Data_batch_1.keys()

dict_keys(['batch_label', 'labels', 'data', 'filenames'])

In [6]:
for item in Data_batch_1:
    print(item, type(Data_batch_1[item]))

batch_label <class 'str'>
labels <class 'list'>
data <class 'numpy.ndarray'>
filenames <class 'list'>


In [57]:
##Labels

In [7]:
Y_train= (Data_batch_1['labels'])
Y_train

[6,
 9,
 9,
 4,
 1,
 1,
 2,
 7,
 8,
 3,
 4,
 7,
 7,
 2,
 9,
 9,
 9,
 3,
 2,
 6,
 4,
 3,
 6,
 6,
 2,
 6,
 3,
 5,
 4,
 0,
 0,
 9,
 1,
 3,
 4,
 0,
 3,
 7,
 3,
 3,
 5,
 2,
 2,
 7,
 1,
 1,
 1,
 2,
 2,
 0,
 9,
 5,
 7,
 9,
 2,
 2,
 5,
 2,
 4,
 3,
 1,
 1,
 8,
 2,
 1,
 1,
 4,
 9,
 7,
 8,
 5,
 9,
 6,
 7,
 3,
 1,
 9,
 0,
 3,
 1,
 3,
 5,
 4,
 5,
 7,
 7,
 4,
 7,
 9,
 4,
 2,
 3,
 8,
 0,
 1,
 6,
 1,
 1,
 4,
 1,
 8,
 3,
 9,
 6,
 6,
 1,
 8,
 5,
 2,
 9,
 9,
 8,
 1,
 7,
 7,
 0,
 0,
 6,
 9,
 1,
 2,
 2,
 9,
 2,
 6,
 6,
 1,
 9,
 5,
 0,
 4,
 7,
 6,
 7,
 1,
 8,
 1,
 1,
 2,
 8,
 1,
 3,
 3,
 6,
 2,
 4,
 9,
 9,
 5,
 4,
 3,
 6,
 7,
 4,
 6,
 8,
 5,
 5,
 4,
 3,
 1,
 8,
 4,
 7,
 6,
 0,
 9,
 5,
 1,
 3,
 8,
 2,
 7,
 5,
 3,
 4,
 1,
 5,
 7,
 0,
 4,
 7,
 5,
 5,
 1,
 0,
 9,
 6,
 9,
 0,
 8,
 7,
 8,
 8,
 2,
 5,
 2,
 3,
 5,
 0,
 6,
 1,
 9,
 3,
 6,
 9,
 1,
 3,
 9,
 6,
 6,
 7,
 1,
 0,
 9,
 5,
 8,
 5,
 2,
 9,
 0,
 8,
 8,
 0,
 6,
 9,
 1,
 1,
 6,
 3,
 7,
 6,
 6,
 0,
 6,
 6,
 1,
 7,
 1,
 5,
 8,
 3,
 6,
 6,
 8,
 6,
 8,
 4,
 6,
 6,


In [ ]:
##Features

In [8]:
X_train = Data_batch_1['data']
X_train

array([[ 59,  43,  50, ..., 140,  84,  72],
       [154, 126, 105, ..., 139, 142, 144],
       [255, 253, 253, ...,  83,  83,  84],
       ...,
       [ 71,  60,  74, ...,  68,  69,  68],
       [250, 254, 211, ..., 215, 255, 254],
       [ 62,  61,  60, ..., 130, 130, 131]], dtype=uint8)

In [9]:
X_train.shape

(10000, 3072)

# PCA using sklearn

In [10]:
pca = PCA()
pca.fit_transform(X_train)

array([[-1.66786333e+03, -7.04951956e+02,  3.55547396e+02, ...,
        -7.67997972e-02, -1.45147111e-01,  7.95970910e-02],
       [ 1.68718819e+02,  2.14158957e+02,  1.55982239e+03, ...,
         1.23589568e-01,  8.57798639e-02,  5.61796128e-02],
       [ 1.96093846e+03,  2.93034428e+03, -6.31292284e+02, ...,
        -1.38117591e-01, -4.95499604e-02,  1.76227162e-01],
       ...,
       [-3.30327785e+02,  2.43792701e+03, -3.74213598e+02, ...,
         1.95087682e-01,  1.00219629e-01,  3.80753848e-01],
       [ 1.79608664e+02, -4.50775000e+02, -4.10255349e+02, ...,
         1.49572559e-01, -1.77296843e-01,  4.56044758e-02],
       [-6.32447973e+02, -1.60751972e+03,  4.78398657e+02, ...,
         2.14303679e-01, -5.54224073e-02,  6.73074833e-02]])

# retaining 90 % variance

In [11]:
# Calculating optimal k

K = 0
total = sum(pca.explained_variance_)
current_sum = 0

while(current_sum / total < 0.90):
    current_sum += pca.explained_variance_[K]
    K += 1
print(K)

96


In [12]:
pca.explained_variance_

array([3.65294943e+06, 1.40842701e+06, 8.19020367e+05, ...,
       6.00529319e-02, 5.91042461e-02, 5.90691286e-02])

In [13]:
## Applying PCA with  calculated K where 90 % variance is retained

pca = PCA(n_components=K, whiten=True, svd_solver='randomized')

X_train = pca.fit_transform(X_train)


SVM Hyperparameter Tuning using GridSearchCV

In [14]:
# defining parameter range
#param_grid = {'C': [0.1, 1, 10, 100, 1000],
              #'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              #'kernel': ['rbf']}

In [15]:
#param_grid = {'C': ['linear'],
              
              #'kernel': ['rbf']}

# Hyperparameter tuning with Grid search CV

In [16]:
param_grid = {'kernel': ('linear', 'rbf'),   ### defining  parameters
              'C': [100, 10, 1]}

In [17]:
grid = GridSearchCV(SVC(), param_grid,refit = True, verbose = 3, n_jobs=8)

In [18]:
grid.fit(X_train, Y_train)  ##fitting the model for grid search

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=SVC(), n_jobs=8,
             param_grid={'C': [100, 10, 1], 'kernel': ('linear', 'rbf')},
             verbose=3)

In [19]:

# printing best parameter after tuning
print(grid.best_params_)
 

{'C': 10, 'kernel': 'rbf'}


In [20]:
# printing the model looks after hyper-parameter tuning
print(grid.best_estimator_)

SVC(C=10)


# Using SVM with the best hyperparameters from above to classify this datasetthrough one vs one (OVO) approach.

In [24]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)
model = SVC(kernel='rbf',C=10,decision_function_shape='ovo')
scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [47]:
scores

array([0.471, 0.463, 0.464, 0.471, 0.47 ])

In [48]:
print('Mean Accuracy: %.3f ' % (mean(scores)))

Mean Accuracy: 0.468 


In [ ]:
##mean class accuracy

In [72]:
y_predict = cross_val_predict(model, X_train,Y_train)

In [73]:
y_predict

array([7, 1, 9, ..., 9, 1, 7])

In [76]:
matrix = confusion_matrix(Y_train, y_predict)
matrix.diagonal()/matrix.sum(axis=1)



array([0.53731343, 0.61190965, 0.35852713, 0.33759843, 0.39139139,
       0.36392743, 0.48349515, 0.51948052, 0.56      , 0.58409786])

# Using SVM with best hyperparameters from above to classify this dataset through one vs rest (OVR) approach.

In [49]:
CV = KFold(n_splits=5, random_state=1, shuffle=True)
Model = SVC(kernel='rbf',C=10,decision_function_shape='ovr')
Scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=CV, n_jobs=-1)

In [51]:
Scores

array([0.471, 0.463, 0.464, 0.471, 0.47 ])

In [52]:
print('Mean Accuracy: %.3f ' % (mean(Scores)))

Mean Accuracy: 0.468 


In [ ]:
##mean class accuracy

In [77]:
Y_predict = cross_val_predict(Model, X_train,Y_train)

In [78]:
Y_predict

array([7, 1, 9, ..., 9, 1, 7])

In [79]:
matrix = confusion_matrix(Y_train, Y_predict)
matrix.diagonal()/matrix.sum(axis=1)



array([0.53731343, 0.61190965, 0.35852713, 0.33759843, 0.39139139,
       0.36392743, 0.48349515, 0.51948052, 0.56      , 0.58409786])